## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


In [1]:
# Your code here - remember to use markdown cells for comments as well!


In [2]:
#Import Packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline

In [3]:
# Loading in Avaition_data.csv
df = pd.read_csv('data/Aviation_Data.csv')

/Users/michaelromanski/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (6,7,28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
# Creating a new DataFrame to keep original intact
aviation_df = df.copy()

In [5]:
# Selecting relevant columns for analysis
aviation_df = aviation_df.drop(['Latitude', 'Longitude', 'Airport.Code', 'Airport.Name', 'Aircraft.Category', 'Publication.Date', 'Air.carrier', 'Broad.phase.of.flight', 'Event.Id', 'Accident.Number', 'Investigation.Type', 'Registration.Number', 'Schedule'], axis=1)
aviation_df                               

,Event.Date,Location,Country,Injury.Severity,Aircraft.damage,Make,Model,Amateur.Built,Number.of.Engines,Engine.Type,FAR.Description,Purpose.of.flight,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Report.Status
0,1948-10-24,"MOOSE CREEK, ID",United States,Fatal(2),Destroyed,Stinson,108-3,No,1.0,Reciprocating,NaN,Personal,2.0,0.0,0.0,0.0,UNK,Probable Cause
1,1962-07-19,"BRIDGEPORT, CA",United States,Fatal(4),Destroyed,Piper,PA24-180,No,1.0,Reciprocating,NaN,Personal,4.0,0.0,0.0,0.0,UNK,Probable Cause
2,1974-08-30,"Saltville, VA",United States,Fatal(3),Destroyed,Cessna,172M,No,1.0,Reciprocating,NaN,Personal,3.0,NaN,NaN,NaN,IMC,Probable Cause
3,1977-06-19,"EUREKA, CA",United States,Fatal(2),Destroyed,Rockwell,112,No,1.0,Reciprocating,NaN,Personal,2.0,0.0,0.0,0.0,IMC,Probable Cause
4,1979-08-02,"Canton, OH",United States,Fatal(1),Destroyed,Cessna,501,No,NaN,NaN,NaN,Personal,1.0,2.0,NaN,0.0,VMC,Probable Cause
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90343,2022-12-26,"Annapolis, MD",United States,Minor,NaN,PIPER,PA-28-151,No,NaN,NaN,091,Personal,0.0,1.0,0.0,0.0,NaN,NaN
90344,2022-12-26,"Hampton, NH",United States,NaN,NaN,BELLANCA,7ECA,No,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN
90345,2022-12-26,"Payson, AZ",United States,Non-Fatal,Substantial,AMERICAN CHAMPION AIRCRAFT,8GCBC,No,1.0,NaN,091,Personal,0.0,0.0,0.0,1.0,VMC,NaN
90346,2022-12-26,"Morgan, UT",United States,NaN,NaN,CESSNA,210N,No,NaN,NaN,091,Personal,0.0,0.0,0.0,0.0,NaN,NaN


In [6]:
# Replacing dot to space in column names
aviation_df = aviation_df.rename(columns=lambda x: x.replace('.', ' '))

In [7]:
# Capitalizing all values in Make
aviation_df['Make'] = aviation_df['Make'].str.capitalize()

In [8]:
# Capitalizing all columns
aviation_df = aviation_df.rename(columns=str.capitalize)

In [9]:
# Changing Event.Date to datetime and creating a year column
aviation_df['Event date'] = pd.to_datetime(aviation_df['Event date'])
aviation_df['Year'] = aviation_df['Event date'].dt.year
aviation_df['Year'] = aviation_df['Year'].astype('Int64')

In [10]:
# Filling all Total Injury-related columns Null data with 0 (the median)
aviation_df.fillna(value={'Total fatal injuries': 0, 'Total serious injuries': 0, 'Total minor injuries': 0, 'Total uninjured': 0}, inplace=True)

In [11]:
#Drop Amateur Built projects because we will not be recommending amateur products to our client. 
aviation_df.drop(aviation_df[aviation_df['Amateur built'] == 'Yes'].index, inplace = True)

In [12]:
#No longer need column since all remaining entries are professionally built
aviation_df = aviation_df.drop(['Amateur built'], axis = 1)

In [13]:
# Drop the values in 'Make' that constitute less than 1% of the total data in 'Make' column
aviation_df = aviation_df[~aviation_df['Make'].isin(aviation_df['Make'].value_counts()[aviation_df['Make'].value_counts() < 84].index)]

In [14]:
# Dropping all rows containing null values from Make, as without a Plane make, the other information is invalid
df.dropna(subset=['Make'], inplace=True)

In [15]:
#Change “Number of engines” nulls to 1, as that is the mode and has a minimal impact on the data distribution
aviation_df['Number of engines'].fillna(1, inplace=True)

In [16]:
aviation_df.dropna(subset = ['Year'], inplace=True)

In [17]:
# Filling in remaining null values in aviation_df with Unknown
aviation_df.fillna('Unknown', inplace=True)

In [ ]:
# WE WILL WORK ON THIS TOMORROW!
df['Location'] = df['Location'].str[-2:]